<a href="https://colab.research.google.com/github/evanolsdal/TimeGAN-Finance-Project/blob/main/TimeGAN_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TimeGAN Training**

This notebook is used to train the timegan model through Google Collab.